In [1]:
import gdal, gdalnumeric, ogr
from PIL import Image, ImageDraw

In [11]:
def imageToArray(i):
    """Converts a Python Imaging Library array to a gdalnumeric image."""
    a=gdalnumeric.numpy.fromstring(i.tobytes(),'b')
    a.shape=i.im.size[1], i.im.size[0]
    return a
def world2Pixel(geoMatrix, x, y):
    """
    Uses a gdal geomatrix (gdal.GetGeoTransform())
    to calculate the pixel location of a
    geospatial coordinate
    """
    ulX = geoMatrix[0]
    ulY = geoMatrix[3]
    xDist = geoMatrix[1]
    yDist = geoMatrix[5]
    rtnX = geoMatrix[2]
    rtnY = geoMatrix[4]
    pixel = int((x - ulX) / xDist)
    line = int((ulY - y) / xDist)
    return (pixel, line)

In [12]:
# Multispectral image used
# to create the NDVI. Must
# have red and infrared
# bands
source = "farm.tif"
# Output geotiff file name
target = "ndvi.tif"
# Load the source data as a gdalnumeric array
srcArray = gdalnumeric.LoadFile(source)
# Also load as a gdal image to
# get geotransform (world file) info
srcImage = gdal.Open(source)
geoTrans = srcImage.GetGeoTransform()
# Red and infrared (or near infrared) bands
r = srcArray[1]
ir = srcArray[2]


In [13]:
## Clip a field out of the bands using a
## field boundary shapefile
# Create an OGR layer from a Field boundary shapefile
field = ogr.Open("field.shp")
# Must define a "layer" to keep OGR happy
lyr = field.GetLayer("field")
# Only one polygon in this shapefile
poly = lyr.GetNextFeature()
# Convert the layer extent to image pixel coordinates
minX, maxX, minY, maxY = lyr.GetExtent()
ulX, ulY = world2Pixel(geoTrans, minX, maxY)
lrX, lrY = world2Pixel(geoTrans, maxX, minY)
# Calculate the pixel size of the new image

In [14]:
pxWidth = int(lrX - ulX)
pxHeight = int(lrY - ulY)
# Create a blank image of the correct size
# that will serve as our mask
clipped = gdalnumeric.numpy.zeros((3, pxHeight, pxWidth), \
                                  gdalnumeric.numpy.uint8)
#mmask = gdalnumeric.zeros((3, pxHeight, pxWidth), gdalnumeric.UnsignedInt8)
#rgb = rgb.astype(gdalnumeric.UnsignedInt8)
rClip = r[ulY:lrY, ulX:lrX]
irClip = ir[ulY:lrY, ulX:lrX]
# Create a new geomatrix for the image
geoTrans = list(geoTrans)
geoTrans[0] = minX
geoTrans[3] = maxY
# Map points to pixels for drawing
# the field boundary on a blank
# 8-bit, black and white, mask image.
points = []
pixels = []
# Grab the polygon geometry
geom = poly.GetGeometryRef()
pts = geom.GetGeometryRef(0)
# Loop through geometry and turn
# the points into an easy-to-manage
# Python list
for p in range(pts.GetPointCount()):
    points.append((pts.GetX(p), pts.GetY(p)))
# Loop through the points and map to pixels.
# Append the pixels to a pixel list
for p in points:
    pixels.append(world2Pixel(geoTrans, p[0], p[1]))
# Create the raster polygon image
rasterPoly = Image.new("L", (pxWidth, pxHeight), 1)
# Create a PIL drawing object
rasterize = ImageDraw.Draw(rasterPoly)
# Dump the pixels to the image
rasterize.polygon(pixels, 0)
# Hand the image back to gdal/gdalnumeric
# so we can use it as an array mask
mask = imageToArray(rasterPoly)


In [15]:
# Clip the red band using the mask
rClip = gdalnumeric.numpy.choose(mask, \
                                 (rClip, 0)).astype(gdalnumeric.numpy.uint8)
# Clip the infrared band using the mask
irClip = gdalnumeric.numpy.choose(mask, \
                                  (irClip, 0)).astype(gdalnumeric.numpy.uint8)

In [16]:
# We don't care about numpy warnings
# due to NaN values from clipping
gdalnumeric.numpy.seterr(all="ignore")
# NDVI equation: (infrared - red) / (infrared + red)
# *1.0 converts values to floats,
# +1.0 prevents ZeroDivisionErrors
ndvi = 1.0 * (irClip - rClip) / irClip + rClip + 1.0
# Remove any NaN values from the final product
ndvi = gdalnumeric.numpy.nan_to_num(ndvi)
# Save ndvi as tiff
gdalnumeric.SaveArray(ndvi, target, \
                      format="GTiff", prototype=source)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x10e38c570> >

In [20]:
import gdalnumeric as gd
import operator
import functools

In [21]:
def histogram(a, bins=range(0,256)):
    """
    Histogram function for multi-dimensional array.
    a = array
    bins = range of numbers to match
    """
    fa = a.flat
    n = gd.numpy.searchsorted(gd.numpy.sort(fa), bins)
    n = gd.numpy.concatenate([n, [len(fa)]])
    hist = n[1:]-n[:-1]
    return hist
def stretch(a):
    """
    Performs a histogram stretch on a gdalnumeric array image.
    """
    hist = histogram(a)
    lut = []
    for b in range(0, len(hist), 256):
        # step size
        step = functools.reduce(operator.add, hist[b:b+256]) / 255 # create equalization lookup table
        n= 0
        for i in range(256):
            lut.append(n / step)
            n = n + hist[i+b]
    gd.numpy.take(lut, a, out=a)
    return a

In [22]:
# NDVI output from ndvi script
source = "ndvi.tif"
# Target file name for classified
# image image
target = "ndvi_color.tif"
# Load the image into an array
ndvi = gd.LoadFile(source).astype(gd.numpy.uint8)
# Peform a histogram stretch so we are able to
# use all of the classes
ndvi = stretch(ndvi)
# Create a blank 3-band image the same size as the ndvi
rgb = gd.numpy.zeros((3, len(ndvi), len(ndvi[0])), gd.numpy.uint8)
# Class list with ndvi upper range values.
# Note the lower and upper values are listed on the ends
classes = [58,73,110,147,184,220,255]
# Color look-up table (lut)
# The lut must match the number of classes
# Specified as R,G,B tuples from dark brown to dark green
lut = [[120,69,25], [255,178,74], [255,237,166], [173,232,94],
          [135,181,64], [3,156,0], [1,100,0]]
# Starting value of the first class
start = 1
# Process all classes.
for i in range(len(classes)):
    mask = gd.numpy.logical_and(start <= ndvi, ndvi <= classes[i])
    for j in range(len(lut[i])):
        rgb[j] = gd.numpy.choose(mask, \
                                 (rgb[j], lut[i][j]))
        start = classes[i]+1
# Save a geotiff of the colorized ndvi.
gd.SaveArray(rgb, target, format="GTiff", prototype=source)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x10e38c150> >

In [23]:
import numpy as np
from linecache import getline
def floodFill(c,r,mask):
    """
    Crawls a mask array containing
    only 1 and 0 values from the
    starting point (c=column,
    r=row - a.k.a. x,y) and returns
    an array with all 1 values
    connected to the starting cell.
    This algorithm performs a 4-way
   check non-recursively.
   """
    # cells already filled
    filled = set()
    # cells to fill
    fill = set()
    fill.add((c,r))
    width = mask.shape[1]-1
    height = mask.shape[0]-1
    # Our output inundation array
    flood = np.zeros_like(mask, dtype=np.int8)
    # Loop through and modify the cells which
    # need to be checked.
    while fill:
        # Grab a cell
        x,y = fill.pop()
        if y == height or x == width or x < 0 or y < 0:
            # Don't fill
            continue
        if mask[y][x] == 1:
            # Do fill
            flood[y][x]=1
            filled.add((x,y))
            # Check neighbors for 1 values
            west =(x-1,y)
            east = (x+1,y)
            north = (x,y-1)
            south = (x,y+1)
            if not west in filled:
                fill.add(west)
            if not east in filled:
                fill.add(east)
            if not north in filled:
                fill.add(north)
            if not south in filled:
                fill.add(south)
    return flood

In [28]:
## MAKING A FLOOD

source = "terrain.asc"
target = "flood.asc"
print ("Opening image...")
img = np.loadtxt(source, skiprows=6)
print ("Image opened")
a = np.where(img<70, 1,0)
print ("Image masked")
# Parse the headr using a loop and
# the built-in linecache module
hdr = [getline(source, i) for i in range(1,7)]
values = [float(h.split(" ")[-1].strip()) for h in hdr]
cols,rows,lx,ly,cell,nd = values
xres = cell
yres = cell * -1
# Starting point for the
# flood inundation
sx = 2582
sy = 2057
print ("Beginning flood fill")
fld = floodFill(sx,sy, a)
print ("Finished Flood fill")
header=""
for i in range(6):
    header += hdr[i]
print ("Saving grid")
# Open the output file, add the hdr, save the array
with open(target, "wb") as f:
    np.savetxt(f, fld, header=header, fmt="%1i")
print ("Done!")

Opening image...
Image opened
Image masked
Beginning flood fill
Finished Flood fill
Saving grid
Done!


In [31]:
## LEAST COST PATHS

import numpy as np
# Width and height # of grids
w= 5
h= 5
# Start location:
# Lower left of grid
start = (h-1, 0)
# End location:
# Top right of grid
dx = w-1
dy = 0
# Blank grid
a = np.zeros((w,h))
# Distance grid
dist = np.zeros(a.shape, dtype=np.int8)
# Calculate distance for all cells
for y,x in np.ndindex(a.shape):
    dist[y][x] = abs((dx-x)+(dy-y))
# "Terrain" is a random value between 1-16.
# Add to the distance grid to calculate
# The cost of moving to a cell
cost = np.random.randint(1,16,(w,h)) + dist
print ("COST GRID (Value + Distance)")
print (cost)

COST GRID (Value + Distance)
[[12 14  4  2  5]
 [10  4 11  2  8]
 [ 8 15  4  6  3]
 [12  6  2 17 12]
 [ 7  9 16  7 14]]


In [33]:
# Our A* search algorithm
def astar(start, end, h, g):
    cset = set()
    oset = set()
    path = set()
    oset.add(start)
    while oset:
        cur = oset.pop()
        if cur == end:
            return path
        cset.add(cur)
        path.add(cur)
        options = []
        y1 = cur[0]
        x1 = cur[1]
        if y1 > 0:
            options.append((y1-1, x1))
        if y1 < h.shape[0]-1:
            options.append((y1+1, x1))
        if x1 > 0:
            options.append((y1, x1-1))
        if x1 < h.shape[1]-1:
            options.append((y1, x1+1))
        if end in options:
            return path
        best = options[0]
        cset.add(options[0])
        for i in range(1,len(options)):
            option = options[i]
            if option in cset:
                continue
            elif h[option] <= h[best]:
                best = option
                cset.add(option)
            elif g[option] < g[best]:
                best = option
                cset.add(option)
            else:
                cset.add(option)
        print (best, ", ", h[best], ", ", g[best])
        oset.add(best)
    return []

In [35]:
print ("(Y,X), HEURISTIC, DISTANCE")
# Find the path
path = astar(start,(dy,dx),cost, dist)
# Create and populate the path grid
path_grid = np.zeros(cost.shape, dtype=np.uint8)
for y,x in path:
    path_grid[y][x]=1
    path_grid[dy][dx]=1
print ("PATH GRID: 1=path")
print (path_grid)

(Y,X), HEURISTIC, DISTANCE
(4, 1) ,  9 ,  1
(3, 1) ,  6 ,  0
(3, 2) ,  2 ,  1
(2, 2) ,  4 ,  0
(2, 3) ,  6 ,  1
(1, 3) ,  2 ,  0
(0, 3) ,  2 ,  1
PATH GRID: 1=path
[[0 0 0 1 1]
 [0 0 0 1 0]
 [0 0 1 1 0]
 [0 1 1 0 0]
 [1 1 0 0 0]]


In [36]:
import numpy as np
import math
from linecache import getline
# Our terrain data
source = "dem2.asc"
# Output file name
# for the path raster
target = "path.asc"
print ("Opening %s..." % source)
cost = np.loadtxt(source, skiprows=6)
print ("Opened %s." % source)
# Parse the header
hdr = [getline(source, i) for i in range(1,7)]
values = [float(ln.split(" ")[-1].strip()) for ln in hdr]
cols,rows,lx,ly,cell,nd = values
# Starting column, row
sx = 1006
sy = 954
# Ending column, row
dx = 303
dy = 109

Opening dem2.asc...
Opened dem2.asc.


In [37]:
def e_dist(p1,p2):
    """
    Takes two points and returns
    the euclidian distance
    """
    x1,y1=p1
    x2,y2=p2
    distance = math.sqrt((x1-x2)**2+(y1-y2)**2)
    return int(distance)
def weighted_score(cur, node, h, start, end):
    """
    Provides a weighted score by comparing the
    current node with a neighboring node. Loosely
    based on the Nisson score concept: f=g+h
    In this case, the "h" value, or "heuristic",
    is the elevation value of each node.
    """
    score = 0
    # current node elevation
    cur_h = h[cur]
    # current node distance from end
    cur_g = e_dist(cur,end)
    # current node distance from
    cur_d = e_dist(cur,start)
    # neighbor node elevation
    node_h = h[node]
    # neighbor node distance from end
    node_g = e_dist(node,end)
    # neighbor node distance from start
    node_d = e_dist(node, start)
    # Compare values with the highest
    # weight given to terrain followed
    # by progress towards the goal.
    if node_h < cur_h:
        score += cur_h-node_h
    if node_g < cur_g:
        score += 10
    if node_d > cur_d:
        score += 10
    return score

In [39]:
def astar(start, end, h):
    """
    A-Star (or A*) search algorithm.
    Moves through nodes in a network
    (or grid), scores each node's
    neighbors, and goes to the node
    with the best score until it finds
    the end. A* is an evolved Dijkstra
    algorithm.
    """
    # Closed set of nodes to avoid
    cset = set()
    # Open set of nodes to evaluate
    oset = set()
    # Output set of path nodes
    path = set()
    # Add the starting point
    # to begin processing
    oset.add(start)
    while oset:
        # Grab the next node
        cur = oset.pop()
        # Return if we're at the end
        if cur == end:
            return path
        # Close off this node to future
        # processing
        cset.add(cur)
        # The current node is always
        # a path node by definition
        path.add(cur)
        # List to hold neighboring
        # nodes for processing
        options = []
        # Grab all of the neighbors
        y1 = cur[0]
        x1 = cur[1]
        if y1 > 0:
            options.append((y1-1, x1))
        if y1 < h.shape[0]-1:
            options.append((y1+1, x1))
        if x1 > 0:
            options.append((y1, x1-1))
        if x1 < h.shape[1]-1:
            options.append((y1, x1+1))
        if x1 > 0 and y1 > 0:
            options.append((y1-1, x1-1))
        if y1 < h.shape[0]-1 and x1 <  h.shape[1]-1:
            options.append((y1+1, x1+1))
        if y1 < h.shape[0]-1 and x1 > 0:
            options.append((y1+1, x1-1))
        if y1 > 0 and x1 <  h.shape[1]-1:
            options.append((y1-1, x1+1))
        # If the end is a neighbor, return
        if end in options:
            return path
        # Store the best known node
        best = options[0]
        # Begin scoring neighbors
        best_score = weighted_score(cur, best, h, start, end)
        # process the other 7 neighbors
        for i in range(1,len(options)):
            option = options[i]
            # Make sure the node is new
            if option in cset:
                continue
            else:
            # Score the option and compare it to the best known
                option_score = weighted_score(cur, option, h, start, end)
            if option_score > best_score:
                best = option
                best_score = option_score
            else:
            # If the node isn't better seal it off
                cset.add(option)
        # Uncomment this print statement to watch
        # the path develop in real time:
        # print best, e_dist(best,end)
        # Add the best node to the open set
        oset.add(best)
    return []

In [41]:
print ("Searching for path...")
p = astar((sy,sx),(dy,dx),cost)
print ("Path found.")
print ("Creating path grid...")
path = np.zeros(cost.shape)
print ("Plotting path...")
for y,x in p:
    path[y][x]=1
    path[dy][dx]=1
print ("Path plotted.")
print ("Saving %s..." % target)
header=""
for i in range(6):
    header += hdr[i]
    # Open the output file, add the hdr, save the array
    with open(target, "wb") as f:
        np.savetxt(f, path, header=header, fmt="%4i")
print ("Done!")

Searching for path...
Path found.
Creating path grid...
Plotting path...
Path plotted.
Saving path.asc...
Done!


In [42]:
#DONE